In [5]:
import pandas as pd
import numpy as np
import sys

# df = pd.DataFrame(new_data, columns=column_names)
# df.set_index('tconst', inplace=True)
# df.to_csv('imdb_years_df.csv', sep=',')

In [11]:
file_path = 'imdb_network.csv'
data = pd.read_csv(file_path)
data = data.values.tolist()

# print(len(data))

# for i in data:
#     print(i)

In [ ]:
class TitleDictionary:

    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path)
        self.title_dict = self._create_title_dict()
        self.profession_dict = self._create_profession_dict()

    def _create_title_dict(self):

        title_dict = {}

        return title_dict 

    def _create_profession_dict(self):

        profession_dict = {}

        return profession_dict
